# Feature Engineering

- **Purpose:** Missing value handling and feature engineering for fraud detection  
- **Author:** Devbrew LLC  
- **Last Updated:** October 18, 2025  
- **Status:** In Progress  
- **License:** Apache 2.0 (Code) | Non-commercial (Data)

---

## Dataset License Notice

This notebook uses the **IEEE-CIS Fraud Detection dataset** from Kaggle.

**Dataset License:** Non-commercial research use only
- You must download the dataset yourself from [Kaggle IEEE-CIS Competition](https://www.kaggle.com/c/ieee-fraud-detection)
- You must accept the competition rules before downloading
- Cannot be used for commercial purposes
- Cannot redistribute the raw dataset

**Setup Instructions:** See [`../data_catalog/README.md`](../data_catalog/README.md) for download instructions.

**Code License:** This notebook's code is licensed under Apache 2.0 (open source).

---

## Notebook Configuration

### Environment Setup

We configure the Python environment with standardized settings, import required libraries, and set a fixed random seed for reproducibility. This ensures consistent results across runs and enables reliable experimentation.

These settings establish the foundation for all feature engineering operations.

In [2]:
import warnings
from pathlib import Path
import json
from typing import Optional, Tuple, List

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)
pd.set_option("display.float_format", '{:.2f}'.format)

# Plotting configuration
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["font.size"] = 10

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("\nEnvironment configurated successfully")
print(f"pandas: {pd.__version__}")
print(f"numpy: {np.__version__}")


Environment configurated successfully
pandas: 2.3.3
numpy: 2.3.3


### Path Configuration

We define the project directory structure and validate that required processed data from the exploration phase exists. The validation ensures we have the necessary inputs before proceeding with feature engineering.

This configuration pattern ensures we can locate all required data artifacts from previous pipeline stages.

In [11]:
# Project paths
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data_catalog"
PROCESSED_DIR = DATA_DIR / "processed"
IEEE_CIS_DIR = DATA_DIR / "ieee-fraud"
NOTEBOOKS_DIR = PROJECT_ROOT / "notebooks"

# Ensure processed directory exists
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Validate required data
def validate_required_data() -> dict:
    """Validate that required datasets exist before feature engineering"""
    paths_status = {
        'IEEE Train Transaction:': (IEEE_CIS_DIR / 'train_transaction.csv').exists(),
        'IEEE Train Identity:': (IEEE_CIS_DIR / 'train_identity.csv').exists(),
    }

    print("\nData Availability Check:")
    for name, exists in paths_status.items():
        status = "Found" if exists else "Missing"
        print(f" • {name} {status}")
    
    all_exist = all(paths_status.values())
    if not all_exist:
        print("\n[WARNING] Some datasets are missing. Check data_catalog/README.md for instructions")
    else:
        print("\nAll required datasets are available")
    
    return paths_status
    
path_status = validate_required_data()


Data Availability Check:
 • IEEE Train Transaction: Found
 • IEEE Train Identity: Found

All required datasets are available
